In [1]:
import os
import json
import time
import torch
import argparse
import numpy as np
from multiprocessing import cpu_count
from torch.utils.data import DataLoader
from collections import OrderedDict, defaultdict
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.utils.rnn as rnn_utils
from utils import to_var, idx2word
from ptb import PTB

In [2]:
ts = time.strftime('%Y-%b-%d-%H-%M-%S', time.gmtime())

In [3]:
save_model_path = os.path.join('won', ts)
os.makedirs('.\\'+save_model_path)

In [4]:
datasets = OrderedDict()

In [5]:
splits = ['train', 'valid'] + (['test'] if False else [])

In [6]:
for split in splits:
    datasets[split] = PTB(
        data_dir='data',
        split=split,
        create_data=False,
        max_sequence_length=60,
        min_occ=1
    )

In [7]:
vocab_size = datasets['train'].vocab_size

sos_idx=datasets['train'].sos_idx

eos_idx=datasets['train'].eos_idx

pad_idx=datasets['train'].pad_idx

In [8]:
max_sequence_length = 60
embedding_size = 300
hidden_size = 256
word_dropout = 0.5
latent_size = 16
num_layers = 1
bidirectional = False
batch_size_fit = 32
rnn_type = 'gru'
learning_rate = 0.0002
k = 0.002
x0 = 2500

In [61]:
class RVAE(nn.Module):
    def __init__(self,vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,
                sos_idx, eos_idx, pad_idx, rnn_type='rnn' , num_layers=1, bidirectional=True):
        
        super().__init__()
        
        self.max_sequence_length = max_sequence_length
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        #self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        self.encoder = Encoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,latent_size = latent_size,rnn_type = rnn_type).cuda()
        self.decoder = Decoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,latent_size = latent_size,rnn_type = rnn_type, word_dropout=word_dropout).cuda()
    
    def forward(self,x,length):        
        batch_size = x.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = x[sorted_idx.cuda()]
        input_embedding = self.embedding(input_sequence)
        
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.tolist(), batch_first=True)

        mu,logvar,reparam = self.encoder(packed_input)
        logp,outputs  = self.decoder(input_embedding, reparam, sorted_lengths, sorted_idx)
        
        
        return logp, mu, logvar, reparam, outputs

In [62]:
class Encoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, latent_size, bidirectional=True, num_layers = 1,rnn_type='rnn'):
        super(Encoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        
        self.encoder = rnn(self.embedding_size, self.hidden_size, num_layers = self.num_layers, bidirectional = self.bidirectional, batch_first = True)
        
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        self.hidden2mean = nn.Linear(self.hidden_size* self.hidden_factor, self.latent_size)
        self.hidden2logv = nn.Linear(self.hidden_size* self.hidden_factor, self.latent_size)
                        
    
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        
        eps = torch.FloatTensor(std.size()).normal_()
        #print(eps)
        eps = Variable(eps).cuda()
        
        return eps.mul(std).add_(mu)
    
    def forward(self,x):
        
        _,hidden = self.encoder(x)
        
        if self.bidirectional or self.num_layers > 1:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size*self.hidden_factor)
        else:
            hidden = hidden.squeeze()

        mu = self.hidden2mean(hidden)
        
        logvar = self.hidden2logv(hidden)
        std = torch.exp(0.5 * logvar)
        #reparam = self.reparametrize(mu,logvar)
        z = Variable(torch.randn([batch_size, self.latent_size])).cuda()
        z = z * std + mu
        
        
        return mu,logvar,z#,reparam

In [73]:
class Decoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, latent_size, bidirectional=True, num_layers = 1,rnn_type='rnn',word_dropout = 0.5):
        super(Decoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers            
        self.latent2hidden = nn.Linear(latent_size, hidden_size * self.hidden_factor)
        
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.word_dropout = nn.Dropout(p=word_dropout)
        
        self.decoder = rnn(embedding_size, hidden_size, num_layers=num_layers, bidirectional=self.bidirectional, batch_first=True)
        self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        
    def forward(self,x,z,sorted_lengths,sorted_idx):

        
        hidden = self.latent2hidden(z)
        if self.bidirectional or self.num_layers > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.hidden_size)
        else:
            hidden = hidden.unsqueeze(0)
        input_embedding = self.word_dropout(x)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.tolist(), batch_first=True)
        outputs,_ = self.decoder(packed_input, hidden)
        
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _,reversed_idx = torch.sort(sorted_idx.cuda())
        padded_outputs = padded_outputs[reversed_idx]
        b,s,_ = padded_outputs.size()
        
        logp = nn.functional.log_softmax(self.outputs2vocab(padded_outputs.view(-1, padded_outputs.size(2))), dim=-1)
        logp = logp.view(b, s, self.embedding.num_embeddings)
        
        return logp,padded_outputs.view(-1, padded_outputs.size(2))

In [74]:
rvae=RVAE(vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,sos_idx, eos_idx, pad_idx , num_layers=num_layers ,rnn_type='gru',bidirectional= False).cuda()

In [75]:
def kl_anneal_function(anneal_function, step, k, x0):
    if anneal_function == 'logistic':
        return float(1/(1+np.exp(-k*(step-x0))))
    elif anneal_function == 'linear':
        return min(1, step/x0)



In [76]:
NLL = torch.nn.NLLLoss(size_average=False, ignore_index=datasets['train'].pad_idx)

In [79]:
def loss_fn(logp, target, length, mean, logv, anneal_function, step, k, x0):

    # cut-off unnecessary padding from target, and flatten
    target = target[:, :torch.max(batch['length'])].contiguous().view(-1)
    logp = logp.view(-1, logp.size(2))

    # Negative Log Likelihood
    NLL_loss = NLL(logp, target)

    # KL Divergence
    KL_loss = -0.5 * torch.sum(1 + logv - mean.pow(2) - logv.exp())
    KL_weight = kl_anneal_function(anneal_function, step, k, x0)

    return NLL_loss, KL_loss, KL_weight

In [80]:
optimizer = torch.optim.Adam(rvae.parameters(), lr=learning_rate)

In [81]:
rvae

RVAE(
  (embedding): Embedding(9877, 300)
  (encoder): Encoder(
    (encoder): GRU(300, 256, batch_first=True)
    (hidden2mean): Linear(in_features=256, out_features=16, bias=True)
    (hidden2logv): Linear(in_features=256, out_features=16, bias=True)
  )
  (decoder): Decoder(
    (latent2hidden): Linear(in_features=16, out_features=256, bias=True)
    (embedding): Embedding(9877, 300)
    (word_dropout): Dropout(p=0.5)
    (decoder): GRU(300, 256, batch_first=True)
    (outputs2vocab): Linear(in_features=256, out_features=9877, bias=True)
  )
)

In [82]:
epochs = 10

In [83]:
step = 0
avg_losses = []
losses = []
NLL_losses = []
KL_losses = []
KL_weights = []

for epoch in range(epochs):
    for split in splits:
        data_loader = DataLoader(
                    dataset=datasets[split],
                    batch_size=batch_size_fit,
                    shuffle=split=='train',
                    num_workers=cpu_count(),
                    pin_memory=torch.cuda.is_available()
                )

        if split == 'train':
            rvae.train()
        else:
            print('********************valid**************')
            rvae.eval()
            
        for iteration, batch in enumerate(data_loader):
            batch_size = batch['input'].size(0)

            x = Variable(batch['input'].type(torch.cuda.LongTensor))
            length = batch['length']
            
            logp, mean, logv, z, outputs=rvae(x,length)
            
            NLL_loss, KL_loss, KL_weight = loss_fn(logp, Variable(batch['target'], volatile=False).type(torch.cuda.LongTensor),batch['length'], mean, logv, 'logistic', step, k, x0)
            
            loss = (NLL_loss + KL_loss*KL_weight)/batch_size#(NLL_loss/batch_size)
            
            
            losses.append(float(loss.cpu().data))
            NLL_losses.append(NLL_loss.data[0]/batch_size)
            KL_losses.append(KL_loss.data[0]/batch_size)
            KL_weights.append(KL_weight)
            
            if split == 'train':
                if iteration % 50 == 0:
                    print('**************backpropagation**********')
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                step += 1

            if iteration % 100 == 0 or iteration+1 == len(data_loader):
                print("%s Batch %04d/%i, Loss %9.4f, NLL-Loss %9.4f, KL-Loss %9.4f, KL-Weight %6.3f"
            %(split.upper(), iteration, len(data_loader)-1, loss.data[0], NLL_loss.data[0]/batch_size, KL_loss.data[0]/batch_size, KL_weight))
                np.savez(L=losses,file='loss.npz')
                np.savez(L=NLL_losses,file='NLL_losses.npz')
                np.savez(L=KL_losses,file='KL_losses.npz') 
                np.savez(L=KL_weights,file='KL_weights.npz')
                
         # save checkpoint
        if split == 'train':
            checkpoint_path = os.path.join(save_model_path, "E%i.pytorch"%(epoch))
            torch.save(rvae, checkpoint_path)
            print("Model saved at %s"%checkpoint_path)
        print("%s Epoch %02d/%i, Mean ELBO %9.4f"%(split.upper(), epoch, epochs, np.mean(np.array(losses))))
        avg_losses.append(np.mean(np.array(losses)))
        np.savez(L=avg_losses,file='avg_losses.npz')

**************backpropagation**********
TRAIN Batch 0000/1314, Loss  213.7115, NLL-Loss  213.7091, KL-Loss    0.3608, KL-Weight  0.007
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0100/1314, Loss  163.8544, NLL-Loss  163.5543, KL-Loss   36.7713, KL-Weight  0.008
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0200/1314, Loss  149.5804, NLL-Loss  149.3199, KL-Loss   26.1708, KL-Weight  0.010
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0300/1314, Loss  123.5468, NLL-Loss  123.2977, KL-Loss   20.5462, KL-Weight  0.012
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0400/1314, Loss  133.0553, NLL-Loss  132.7648, KL-Loss   19.6581, KL-Weight  0.015
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0500/1314, Loss  141.5441, NLL-Loss  141.0626, KL-Loss   26.7660, KL-Weig

C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type RVAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\asd13\AppData\Local\Continuum\Anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model saved at won\2018-May-19-11-06-12\E0.pytorch
TRAIN Epoch 00/10, Mean ELBO  138.8191
********************valid**************
VALID Batch 0000/105, Loss  141.2753, NLL-Loss  139.5719, KL-Loss   19.9247, KL-Weight  0.085
VALID Batch 0100/105, Loss  106.0866, NLL-Loss  104.6399, KL-Loss   16.9228, KL-Weight  0.085
VALID Batch 0105/105, Loss  110.5615, NLL-Loss  108.3503, KL-Loss   25.8654, KL-Weight  0.085
VALID Epoch 00/10, Mean ELBO  137.7942
**************backpropagation**********
TRAIN Batch 0000/1314, Loss  125.6303, NLL-Loss  123.7312, KL-Loss   22.2150, KL-Weight  0.085
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0100/1314, Loss  119.9919, NLL-Loss  117.7887, KL-Loss   21.5000, KL-Weight  0.102
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0200/1314, Loss  133.8987, NLL-Loss  131.4926, KL-Loss   19.6597, KL-Weight  0.122
**************backpropagation**********
**************backpropag

**************backpropagation**********
**************backpropagation**********
TRAIN Batch 1100/1314, Loss  122.5504, NLL-Loss  119.5932, KL-Loss    2.9754, KL-Weight  0.994
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 1200/1314, Loss  117.1365, NLL-Loss  114.4724, KL-Loss    2.6775, KL-Weight  0.995
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 1300/1314, Loss  129.6980, NLL-Loss  126.8619, KL-Loss    2.8479, KL-Weight  0.996
TRAIN Batch 1314/1314, Loss  111.9205, NLL-Loss  109.4596, KL-Loss    2.4708, KL-Weight  0.996
Model saved at won\2018-May-19-11-06-12\E3.pytorch
TRAIN Epoch 03/10, Mean ELBO  126.2404
********************valid**************
VALID Batch 0000/105, Loss  135.7032, NLL-Loss  133.3535, KL-Loss    2.3591, KL-Weight  0.996
VALID Batch 0100/105, Loss  102.2436, NLL-Loss  100.1126, KL-Loss    2.1396, KL-Weight  0.996
VALID Batch 0105/105, Loss  103.3240, NLL-Loss  100.2538, KL-L

**************backpropagation**********
TRAIN Batch 0700/1314, Loss  118.6126, NLL-Loss  115.3353, KL-Loss    3.2773, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0800/1314, Loss  115.3015, NLL-Loss  112.6812, KL-Loss    2.6203, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0900/1314, Loss   96.9234, NLL-Loss   94.3801, KL-Loss    2.5432, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 1000/1314, Loss  105.9353, NLL-Loss  102.9306, KL-Loss    3.0047, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 1100/1314, Loss  129.8645, NLL-Loss  126.5664, KL-Loss    3.2982, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 1200/1314, Loss  105.1134, NLL-Loss  102.1760, KL-Loss    2.9374, KL-Weig

**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0400/1314, Loss  116.1655, NLL-Loss  112.3220, KL-Loss    3.8435, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0500/1314, Loss   93.2400, NLL-Loss   90.1964, KL-Loss    3.0435, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0600/1314, Loss  107.7135, NLL-Loss  104.4114, KL-Loss    3.3021, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0700/1314, Loss   94.0198, NLL-Loss   91.1826, KL-Loss    2.8373, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0800/1314, Loss  109.0472, NLL-Loss  105.2819, KL-Loss    3.7653, KL-Weight  1.000
**************backpropagation**********
**************backpropagation**********
TRAIN Batch 0900/1314, Loss  109.5888, NLL-Lo

In [105]:
def inference(batch, z):
    
    hidden = rvae.decoder.latent2hidden(z)
    hidden = hidden.view(rvae.decoder.hidden_factor, batch_size, rvae.hidden_size)
    
    t = 0
    generations=torch.cuda.FloatTensor(batch_size, max_sequence_length).fill_(pad_idx).long()
    
    while(t<max_sequence_length ):
        if t == 0:
            input_sequence = Variable(torch.Tensor(batch_size).fill_(sos_idx).long()).cuda()
        input_sequence = input_sequence.unsqueeze(1)
        input_embedding = rvae.embedding(input_sequence)
        output, hidden = rvae.decoder.decoder(input_embedding, hidden)
        logits = rvae.decoder.outputs2vocab(output)
        samples = torch.topk(logits,1,dim=-1)[1].squeeze()
        input_sequence = samples.squeeze()
        generations[:,t] = input_sequence.data
        if ((torch.sum(input_sequence)/input_sequence.size(0)) == eos_idx).cpu().data.numpy():
            break
        t+=1
    return generations

def print_inference(generations):
    with open('data'+'/ptb.vocab.json', 'r') as file:
        vocab = json.load(file)

    w2i, i2w = vocab['w2i'], vocab['i2w']

    samples = generations.cpu().numpy()

    sent_str = [str()]*len(samples)

    for i, sent in enumerate(samples):
        for word_id in sent:
            if word_id == w2i['<pad>']: 
                break
            sent_str[i] += i2w[str(word_id)] + " "
        sent_str[i] = sent_str[i].strip()
    return sent_str

In [106]:
try:
    rvae = torch.load(checkpoint_path)
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass
rvae.cuda()
rvae.eval()


--------model restored--------



RVAE(
  (embedding): Embedding(9877, 300)
  (encoder): Encoder(
    (encoder): GRU(300, 256, batch_first=True)
    (hidden2mean): Linear(in_features=256, out_features=16, bias=True)
    (hidden2logv): Linear(in_features=256, out_features=16, bias=True)
  )
  (decoder): Decoder(
    (latent2hidden): Linear(in_features=16, out_features=256, bias=True)
    (embedding): Embedding(9877, 300)
    (word_dropout): Dropout(p=0.5)
    (decoder): GRU(300, 256, batch_first=True)
    (outputs2vocab): Linear(in_features=256, out_features=9877, bias=True)
  )
)

In [135]:
steps = 8

z1 = torch.randn([latent_size]).numpy()
z2 = torch.randn([latent_size]).numpy()

interpolation = np.zeros((z1.shape[0], steps + 2))

for dim, (s,e) in enumerate(zip(z1,z2)):
    interpolation[dim] = np.linspace(s,e,steps+2)
    
z = Variable(torch.from_numpy(interpolation.T).float()).cuda()

In [136]:
batch_size = z.size(0)

In [137]:
generations=inference(batch_size, z)

In [138]:
sent_str = print_inference(generations)

In [139]:
for i in sent_str:
    print(i)

mr . n said it will be a n n stake in the company ' s n <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
mr . n said it will be a n n stake in the company ' s n <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
mr . n said it will be a n n stake in the n of the company ' s n <eos> <eos> <eos> <eos> <eos>
mr . n said that the company ' s n n of the company ' s n n <eos> <eos> <eos> <eos> <eos> <eos> <eos>
mr . n said that the company ' s n n of the n n n n <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
mr . n said that the n n of the n n n n of the n n n n <eos> <eos> <eos> <eos> <eos>
but the n of the n n n n of the n n n n n n n and n n of the n n
but the n of the n n n of the n n n n n n n and n n of the n n n
but the n of the n n n of the n n n n of the n n n n n n in the n
but the n of the n n of the n n n of the n n n n of the n n n n n


In [161]:
batch_size = 5

z = Variable(torch.randn([batch_size, latent_size])).cuda()

In [162]:
generations=inference(batch_size, z)

In [163]:
sent_str = print_inference(generations)

In [164]:
for i in sent_str:
    print(i)

the company said it is a n n stake in the n <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
but he says he is a n of the n n of the n of the n of the n
the company is a n n stake in n <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
the company said it will be a n n stake in the u . s . and n <eos> <eos>
i ' m not going to be n <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
